# Summary

`# TODO: summary here`

In [1]:
%load_ext autoreload
%autoreload 3

In [93]:
from abc import ABC, abstractmethod
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, \
    PegasusTokenizerFast, pipeline

from htools import *
from incendio.utils import DEVICE, gpu_setup

In [12]:
gpu_setup(False)

cpu


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/hmamin/anaconda3/lib/python3.7/site-packages/incendio/utils.py:38: UserWarning: Cuda not available.
  if not torch.cuda.is_available(): warnings.warn('Cuda not available.')
/Users/hmamin/anaconda3/lib/python3.7/site-packages/incendio/utils.py:41: UserWarning: Incendio device is not cuda.
  warnings.warn('Incendio device is not cuda.')


In [2]:
version = 'tuner007/pegasus_paraphrase'
net = PegasusForConditionalGeneration.from_pretrained(version)
tok = PegasusTokenizerFast.from_pretrained(version)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/Users/hmamin/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.1.119', 60954), raddr=('52.216.233.221', 443)>
  self._sock = None
/Users/hmamin/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.1.119', 60955), raddr=('52.216.233.221', 443)>
  self._sock = None


/Users/hmamin/anaconda3/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:69: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/hmamin/.cache/torch/transformers/fa4532c0035b101d7abcd5c0c9c34a83288902b66c5616034db1a47643e05c75.efce77b8dcd2c57b109b0d10170fcdcd53f23c21286974d4f66706536758ab6e'>
  m.ParseFromString(open(filename, "rb").read())


In [39]:
_ = net.to(DEVICE)

In [50]:
texts = [
    'Educational games and digital learning materials to provide K-12 '
    'students with enriching experiences.',
    'The world\'s largest social network. Helping people build and maintain '
    'relationships in a disconnected world.',
    'I hate school. I wish my teacher would leave me alone. I don\'t think '
    ' he likes me.',
    'Today the president announced new plans to revamp the private '
    'healthcare system. Pundits questioned how he would manage to pass the '
    'bill.'
]

In [5]:
hdir(tok)

Using bos_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.


{'SPECIAL_TOKENS_ATTRIBUTES': 'attribute',
 'add_special_tokens': 'method',
 'add_tokens': 'method',
 'additional_special_tokens': 'attribute',
 'additional_special_tokens_ids': 'attribute',
 'all_special_ids': 'attribute',
 'all_special_tokens': 'attribute',
 'all_special_tokens_extended': 'attribute',
 'backend_tokenizer': 'attribute',
 'batch_decode': 'method',
 'batch_encode_plus': 'method',
 'bos_token': 'attribute',
 'bos_token_id': 'attribute',
 'build_inputs_with_special_tokens': 'method',
 'clean_up_tokenization': 'method',
 'cls_token': 'attribute',
 'cls_token_id': 'attribute',
 'convert_ids_to_tokens': 'method',
 'convert_tokens_to_ids': 'method',
 'create_token_type_ids_from_sequences': 'method',
 'decode': 'method',
 'deprecation_warnings': 'attribute',
 'encode': 'method',
 'encode_plus': 'method',
 'eos_token': 'attribute',
 'eos_token_id': 'attribute',
 'from_pretrained': 'method',
 'get_added_vocab': 'method',
 'get_special_tokens_mask': 'method',
 'get_vocab': 'metho

In [6]:
res = tok.prepare_seq2seq_batch(texts, truncation=True, padding='longest')

In [7]:
res

{'input_ids': tensor([[11263,   727,   111,  1016,   761,   917,   112,   319,  1046,  6054,
           392,   122, 26838,  1747,   107,     1,     0,     0,     0,     0,
             0,     0,     0],
        [  139,   278,   131,   116,  1368,   525,   952,   107, 22844,   200,
           736,   111,  1634,  2074,   115,   114, 20402,   278,   107,     1,
             0,     0,     0],
        [  125,  4180,   399,   107,   125,  1216,   161,  2118,   192,   858,
           213,  1600,   107,   125,   272,   131,   144,   311,   178,  5606,
           213,   107,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
gen = net.generate(**res)

tensor([[   0, 1046, 6054,  392,  137,  207, 2387,  727,  111, 1016,  761,  917,
          107,    1,    0,    0,    0],
        [   0,  139,  278,  131,  116, 1368,  525,  952,  107,    1,    0,    0,
            0,    0,    0,    0,    0],
        [   0,  125,  272,  131,  144,  172,  399,  111,  161, 2118,  591,  131,
          144,  172,  213,  107,    1]])

In [19]:
tok.batch_decode(gen.tolist(), skip_special_tokens=True)

['K-12 students can use educational games and digital learning materials.',
 "The world's largest social network.",
 "I don't like school and my teacher doesn't like me."]

In [54]:
@add_docstring(net.generate)
def paraphrase(text, n=1, temperature=1.5, **gen_kwargs):
    batch = tok.prepare_seq2seq_batch([text], truncation=True, 
                                      padding='longest').to(DEVICE)
    paraphrased = net.generate(**batch, num_return_sequences=n,
                               temperature=temperature, **gen_kwargs)
    return tok.batch_decode(paraphrased.tolist(), skip_special_tokens=True)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
paraphrase(texts[3], n=5, num_beams=10)

['The president announced plans to reform the private healthcare system.',
 'Pundits wondered how the president would pass the bill.',
 'Pundits questioned how the president would pass the bill.',
 'Pundits were questioning how the president would pass the bill.',
 'Pundits questioned how he would get the bill passed.']

In [53]:
texts[-1]

'Today the president announced new plans to revamp the private healthcare system. Pundits questioned how he would manage to pass the bill.'

In [48]:
net.generate()

{'input_ids': <Parameter "input_ids: Union[torch.LongTensor, NoneType] = None">,
 'decoder_input_ids': <Parameter "decoder_input_ids: Union[torch.LongTensor, NoneType] = None">,
 'max_length': <Parameter "max_length: Union[int, NoneType] = None">,
 'min_length': <Parameter "min_length: Union[int, NoneType] = None">,
 'do_sample': <Parameter "do_sample: Union[bool, NoneType] = None">,
 'early_stopping': <Parameter "early_stopping: Union[bool, NoneType] = None">,
 'num_beams': <Parameter "num_beams: Union[int, NoneType] = None">,
 'temperature': <Parameter "temperature: Union[float, NoneType] = None">,
 'top_k': <Parameter "top_k: Union[int, NoneType] = None">,
 'top_p': <Parameter "top_p: Union[float, NoneType] = None">,
 'repetition_penalty': <Parameter "repetition_penalty: Union[float, NoneType] = None">,
 'bad_words_ids': <Parameter "bad_words_ids: Union[Iterable[int], NoneType] = None">,
 'bos_token_id': <Parameter "bos_token_id: Union[int, NoneType] = None">,
 'pad_token_id': <Para

In [57]:
class ParaphrasePipeline:
    
    def __init__(self):
        self.name = 'tuner007/pegasus_paraphrase'
        self.net = PegasusForConditionalGeneration.from_pretrained(self.name)\
                                                  .to(DEVICE)
        self.tok = PegasusTokenizerFast.from_pretrained(self.name)
        
    @add_docstring(PegasusForConditionalGeneration.generate)
    def __call__(self, text, n=1, temperature=1.5, **kwargs):
        batch = self.tok.prepare_seq2seq_batch([text], truncation=True, 
                                               padding='longest').to(DEVICE)
        gen_tokens = self.net.generate(**batch, num_return_sequences=n,
                                       temperature=temperature, **kwargs)
        return self.tok.batch_decode(gen_tokens.tolist(),
                                     skip_special_tokens=True)

In [56]:
pipe = ParaphrasePipeline()

/Users/hmamin/anaconda3/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:69: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/hmamin/.cache/torch/transformers/fa4532c0035b101d7abcd5c0c9c34a83288902b66c5616034db1a47643e05c75.efce77b8dcd2c57b109b0d10170fcdcd53f23c21286974d4f66706536758ab6e'>
  m.ParseFromString(open(filename, "rb").read())


In [58]:
pipe('It was a beautiful rainy day and every falling footstep splattered '
     'water into Arnold\'s face.')

["Arnold's face was covered in water after every falling footstep."]

In [63]:
class TransformerTransform:
    
    def __init__(self, mode, n=1):
        self.mode = mode
        self.n = n
        self.predictor = self._get_transformer()
        
#     def __call__(self, text):
#         self._preprocess(text)
    
#     @abstractmethod
#     def _preprocess(self, text):
#         pass
    
    def _get_transformer(self):
        if self.mode == 'mask':
            return pipeline('fill-mask')
        elif self.mode == 'generate':
            return pipeline('text-generation')
        elif self.mode == 'paraphrase':
            return ParaphrasePipeline()
        else:
            raise ValueError('mode must be in (mask, generate, paraphrase).')

In [64]:
MASK = '<mask>'

In [104]:
class FillMaskTransform:    
    
    def __init__(self):
        self.pipe = pipeline('fill-mask')
    
    def _preprocess(self, text, n=1, min_keep=3):
        tokens = text.split()
        idx = np.random.choice(range(len(tokens)), 
                               min(len(tokens)-min_keep, n),
                               replace=False)
        return ' '.join(t if i not in idx else MASK 
                        for i, t in enumerate(tokens))
    
    def __call__(self, text, n=1, return_all=False):
        # Each item will be a list of strings. Each string in res[i]
        # will have i words changed.
        res = [[text]]
        for i in range(n+1):
            text = [enc['sequence'].replace('<s>', '').replace('</s>', '')
                    for row in res[-1] for enc in 
                    self.pipe(self._preprocess(row))]
            res.append(text)
        if not return_all: res = res[n]
        return res

In [105]:
t = 'I went to the store today to buy eggs.'

In [106]:
masker = FillMaskTransform()

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [107]:
masker(t)

['I went to the store today to buy eggs.',
 'I headed to the store today to buy eggs.',
 'I drove to the store today to buy eggs.',
 'I came to the store today to buy eggs.',
 'I head to the store today to buy eggs.']

In [108]:
masker(t, 3)

['I went to the store today to buy eggs.',
 'I went to the store today and buy eggs.',
 'I went to the store today & buy eggs.',
 'I went to the store today I buy eggs.',
 'I went to the store today, buy eggs.',
 'I went to grocery store today to purchase eggs.',
 'I went to the store today to purchase eggs.',
 'I went to my store today to purchase eggs.',
 'I went to a store today to purchase eggs.',
 'I went to another store today to purchase eggs.',
 'I went to the store today to buy eggs.',
 'I went to the store today to purchase eggs.',
 'I went to the store today to get eggs.',
 'I went to the store today to collect eggs.',
 'I went to the store today to grab eggs.',
 'I went to the store today to buy eggs.',
 'I went to the store today to purchase eggs.',
 'I went to the store today to get eggs.',
 'I went to the store today to collect eggs.',
 'I went to the store today to grab eggs.',
 'I went to the store today to grab groceries',
 'I went to the store today to grab some',
 '

In [110]:
masker(t, 2, return_all=True)

[['I went to the store today to buy eggs.'],
 ['I went to the store today to buy eggs.',
  'I went to the store today to purchase eggs.',
  'I went to the store today to get eggs.',
  'I went to the store today to collect eggs.',
  'I went to the store today to grab eggs.'],
 ['I went to the store today to buy eggs.',
  'We went to the store today to buy eggs.',
  'She went to the store today to buy eggs.',
  'He went to the store today to buy eggs.',
  'Someone went to the store today to buy eggs.',
  'I went to the store today to buy eggs.',
  'I went to the store today to purchase eggs.',
  'I went to the store today to get eggs.',
  'I went to the store today to collect eggs.',
  'I went to the store today to grab eggs.',
  'I went to the store today to get eggs.',
  'I went into the store today to get eggs.',
  'I went through the store today to get eggs.',
  'I went around the store today to get eggs.',
  'I went in the store today to get eggs.',
  'I went to the store today to c

In [111]:
masker(t)

['I went to grocery store today to buy eggs.',
 'I went to the store today to buy eggs.',
 'I went to my store today to buy eggs.',
 'I went to a store today to buy eggs.',
 'I went to another store today to buy eggs.']

In [114]:
masker.pipe('I went to work after <mask>.')

[{'sequence': '<s>I went to work after school.</s>',
  'score': 0.21024835109710693,
  'token': 334,
  'token_str': 'Ġschool'},
 {'sequence': '<s>I went to work after graduation.</s>',
  'score': 0.10130680352449417,
  'token': 11412,
  'token_str': 'Ġgraduation'},
 {'sequence': '<s>I went to work after lunch.</s>',
  'score': 0.09977195411920547,
  'token': 4592,
  'token_str': 'Ġlunch'},
 {'sequence': '<s>I went to work after midnight.</s>',
  'score': 0.06013602390885353,
  'token': 5832,
  'token_str': 'Ġmidnight'},
 {'sequence': '<s>I went to work after dinner.</s>',
  'score': 0.04950949549674988,
  'token': 3630,
  'token_str': 'Ġdinner'}]